# HyperStream Tutorial 1: Introduction

## Requirements

In order to run this and the following tutorials, it is necessary to have access to a MongoDB server running in the **localhost port 27017**. It is possible to change the host and port of the MongoDB server by modifying the configuration file __hyperstream_config.json__ located in the same folder as this notebook.

We also require all the dependencies listed in the HyperStream requirements, the installation instructions can be found in https://github.com/IRC-SPHERE/HyperStream

In [1]:
import sys
sys.path.append("../") # Add parent dir in the Path

from hyperstream import HyperStream

## Starting a Hyperstream instance

First of all, we will create a HyperStream instance. This instance will connect to the MongoDB server that is specified in the configuration file and it will raise an error if the MongoDB server is not running.

In [2]:
from hyperstream import HyperStream

hs = HyperStream(loglevel=0)
print hs

HyperStream version 0.2.6, connected to mongodb://localhost:27017/hyperstream


## Selecting a tool

HyperStream counts with a set of predefined tools in hyperstream.tools. This tools can be used to define the nodes of a factor graph that will produce values or compute certain functions given the specified input nodes. For this tutorial, we will focus on the **clock** tool. This tool produces time ticks from the specified start and stride times.

In [3]:
from hyperstream import StreamId

T = hs.channel_manager.tools

clock = StreamId(name="clock")

clock_tool = T[clock].window().last().value(stride=2.0)

## Specifying the memory channel

We need to specify where do we want to store the resulting stream of data that will be generated. It is possible to choose a MongoDB database instead of in memory selecting **hs.channel_manger.mongo**. In this tutorial we use the memory channel by creating an instance of memory and then creating the stream on it.

In [4]:
M = hs.channel_manager.memory

ticker = M.get_or_create_stream(stream_id=StreamId(name="ticker"))

## Querying the tool

Now we only need to create the time interval that we want to query. We do this by specifieng the begining and end.

In [5]:
from pytz import UTC
from datetime import datetime, timedelta

now = datetime.utcnow().replace(tzinfo=UTC)
before = (now - timedelta(seconds=10)).replace(tzinfo=UTC)

from hyperstream import TimeInterval

ti = TimeInterval(before, now)

## Executing the tool

Now that we defined the tool to use, where we want to store the results and the time interval, it is possible to execute the tool.

In [6]:
clock_tool.execute(sources=[], sink=ticker, interval=ti, alignment_stream=None)

## Printing the results

The resulting stream is stored in the ticker. We can get now a list of tuples containing the timestamps and its corresponding clock value.

In [7]:
for timestamp, value in ticker.window().items():
    print '[%s]: %s' % (timestamp, value)

[2017-06-19 15:51:16+00:00]: 2017-06-19 15:51:16+00:00
[2017-06-19 15:51:18+00:00]: 2017-06-19 15:51:18+00:00
[2017-06-19 15:51:20+00:00]: 2017-06-19 15:51:20+00:00
[2017-06-19 15:51:22+00:00]: 2017-06-19 15:51:22+00:00
[2017-06-19 15:51:24+00:00]: 2017-06-19 15:51:24+00:00
